In [3]:
!pip install keras

In [4]:
!pip install tensorflow

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import*
from keras.models import * 
# shift+ tab for tooltip

In [6]:
from keras.preprocessing import image

In [7]:
#CNN based model in keras 
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
# Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3) akhane 32 hochee filters , kernel size (3,3) mane 
# hochhe 3x3 matrix or window input shape (224,224,3) mane hochhe width and height 224 and no of channel 3 
model.add(Conv2D(64,(3,3),activation='relu')) # 2 ta convolution layer add korsi karon non linearity handle kora jabe  # reduce the number of parameters 
model.add(MaxPooling2D(pool_size=(2,2)))      
model.add(Dropout(0.25))    # dropout for finding overfitting

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))  # As  Iwill go to the deeper of network I will increase the number of filters like 128 here
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu')) # dense layer output layer
model.add(Dropout(0.25))
model.add(Dense(1,activation='sigmoid')) # single neuron in output layer


model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

# as I go into deeper I would increase number of channels here I first decalred 32 filters 
# then 64 and finally 128. here channel means receptive field 
# If we look at a object from large distance we wil lfind more complex patterns in that object 

          

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 64)        0

In [9]:
# train from scratch
train_datagen = image.ImageDataGenerator(
    
    rescale = 1./255, # normalization
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,


)

test_dataset = image.ImageDataGenerator(rescale = 1./255)


In [10]:
train_generator = train_datagen.flow_from_directory(
    'Dataset/Train',
    target_size =(224,224),
    batch_size =32,
    class_mode = 'binary'

)

Found 392 images belonging to 2 classes.


In [11]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [12]:
validation_generator = test_dataset.flow_from_directory(
    'Dataset/Test',
    target_size =(224,224),
    batch_size =32,
    class_mode = 'binary'

)

Found 80 images belonging to 2 classes.


In [13]:
validation_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [14]:
import tensorflow as tf
tf.version.VERSION


'2.3.1'

In [15]:

import keras
keras.__version__

'2.4.3'

In [16]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch =8,
    epochs=15,
    validation_data = validation_generator,
    validation_steps = 2
    
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
8/8 [==============================] - 19s 2s/step - loss: 0.7526 - accuracy: 0.6172 - val_loss: 0.6066 - val_accuracy: 0.5156
Epoch 2/15
8/8 [==============================] - 18s 2s/step - loss: 0.4827 - accuracy: 0.7930 - val_loss: 0.3804 - val_accuracy: 0.9062
Epoch 3/15
8/8 [==============================] - 17s 2s/step - loss: 0.3173 - accuracy: 0.8707 - val_loss: 0.2113 - val_accuracy: 0.9531
Epoch 4/15
8/8 [==============================] - 18s 2s/step - loss: 0.1580 - accuracy: 0.9336 - val_loss: 0.1025 - val_accuracy: 0.9688
Epoch 5/15
8/8 [==============================] - 16s 2s/step - loss: 0.2100 - accuracy: 0.9310 - val_loss: 0.1488 - val_accuracy: 0.9688
Epoch 6/15
8/8 [==============================] - 19s 2s/step - loss: 0.1242 - accuracy: 0.9612 - val_loss: 0.1110 - val_accuracy: 0.9688
Epoch 7/15
8/8 [==============================] - 17s 2s/step - loss: 0.1490 - accuracy: 0.9526 

Loss calculation

In [18]:
# try to use transfer learning -> imagnet # need more images like 1-2 k images  
# using gradcam, class activation maping we can extract the roi  

In [19]:
model.evaluate_generator(train_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.22174802422523499, 0.9438775777816772]

In [20]:
model.evaluate_generator(validation_generator)

[0.16934525966644287, 0.9750000238418579]

In [21]:
# save model 
model.save('CNNModel.h')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: CNNModel.h\assets


HeatMap on test Data

In [22]:
model = load_model('CNNModel.h')
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [24]:
y_actual = []
y_test = []

In [26]:
import os
for i in os.listdir("./Dataset/Test/Normal/"):
    img = image.laod_img("./Dataset/Test/Normal/" +i,target_size=(224,224))
    img = image.img_to_array(img)
    img= np.expand_dims(img,axis=0)
    p=model.predict_classes(img)
    y_test.append(p[0,0])
    y_actual.append(1)

AttributeError: module 'tensorflow.keras.preprocessing.image' has no attribute 'laod_img'